<center> <h1>PRIMERA PREDICIÓN DEL SUEÑO DE UN PACIENTE</h1> </center>
<center> <h4>Noemi González, Roberto Holgado y Carmen Plaza seco</h4> </center>

[URL PHYSIONET](https://physionet.org/content/sleep-accel/1.0.0/)

###### Indice

* **1. Preprocesado, extracción de características.**
  *   **1.1 Extracción variables de frecuencia cardíaca**
  *  **1.2 Extracción variables de movimiento**
  *  **1.3 Obtención matrices**
* **2. Conjuntos Train y Test**
  * **2.1 Preparación datos de entreada y salida**
* **3. Clasificador SVM Binario**
   * **3.1 Creación del modelo y selección parámetros libres**
   * **3.2 Evaluación del modelo**
* **4. Clasficador SVM Multiclase**
   * **4.1 Creación del modelo y selección parámetros libres**
      *     **4.2.1**
      *     **4.2.2**
   * **4.3 Evaluación del modelo**

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import statistics as stats
from sklearn.model_selection import GridSearchCV
from random import sample
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV

## 1. CARGAR DATOS

### 2.1 Preparación datos de entrada y salida

In [2]:
#Train
Train= pd.read_csv('Train.csv').drop(['Unnamed: 0'], axis = 1)
Test= pd.read_csv('Test.csv').drop(['Unnamed: 0'], axis = 1)

In [3]:
#Test

In [4]:
X_train=Train.drop(['Etiquetas', 'Etiquetas binarias'], axis = 1) 
y_train_bin=Train['Etiquetas binarias']
y_train_multi=Train['Etiquetas']

X_test=Test.drop(['Etiquetas', 'Etiquetas binarias'], axis = 1)
y_test_bin=Test['Etiquetas binarias']
y_test_multi=Test['Etiquetas']

In [5]:
X_train

,Min_hr,Max_hr,Mean_hr,Min_acc,Max_acc,Mean_acc,ID_P
0,72.0,77.0,73.33,0.98,1.02,0.99,19.0
1,70.0,75.0,71.60,0.75,1.39,1.00,19.0
2,71.0,75.0,72.29,0.91,1.05,0.99,19.0
3,70.0,72.0,71.20,0.84,1.13,0.99,19.0
4,68.0,70.0,68.80,0.99,1.00,0.99,19.0
...,...,...,...,...,...,...,...
17941,73.0,74.0,73.40,1.00,1.01,1.00,4.0
17942,74.0,76.0,74.80,0.99,1.01,1.00,4.0
17943,71.0,76.0,73.33,0.99,1.01,1.00,4.0
17944,74.0,77.0,75.71,1.00,1.01,1.00,4.0


## 3. Clasficador SVM Binario


### 3.1  Creación del modelo y selección parámetros libres

In [6]:
clf = SVC()
modelo_base=clf.fit(X_train,y_train_bin)

C:\Users\rober\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [7]:
clf

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [8]:
Cs =[1, 5]  #list(range(0.001,10)
param_grid = {'kernel':('linear', 'rbf'),'C': Cs}
svc_grid = GridSearchCV(clf, param_grid, scoring='accuracy',cv=3,n_jobs=-1)
svc_grid.fit(X_train, y_train_bin)
svc_grid.best_params_

{'C': 5, 'kernel': 'linear'}

In [9]:
svc_clf=svc_grid.best_estimator_

In [10]:
print("Tasa de acierto del modelo base:",modelo_base.score(X_test,y_test_bin))
print("Tasa de acierto con la busqueda de parámetros:",svc_clf.score(X_test,y_test_bin))

#Mejora
print('Mejora  del {:0.2f}%.'.format( 100 * (svc_clf.score(X_test,y_test_bin) - modelo_base.score(X_test,y_test_bin)) / modelo_base.score(X_test,y_test_bin)))

Tasa de acierto del modelo base: 0.5138434432418827
Tasa de acierto con la busqueda de parámetros: 0.7387364711804681
Mejora  del 43.77%.


### 3.3 Evaluación del modelo

In [11]:
predict = svc_clf.predict(X_test)

#EVALUACION
acc_test= svc_clf.score(X_test, y_test_bin).round(4)
acc_train= svc_clf.score(X_train, y_train_bin).round(4)
print('La exactitud para el modelo de SVM en el conjunto de TEST es:',acc_test,'\n')
print('La exactitud para el modelo de SVM en el conjunto de TRAIN es:',acc_train,'\n')

print('Confusion Matrix')
print(confusion_matrix(y_test_bin, predict),'\n')

print("=============== Classification Report ================")
print(classification_report(y_test_bin, predict))
#matriz_confusion=pd.crosstab(Y_test,predict, rownames=['Clase predicha'], colnames=['Resultado del clasificador'])

La exactitud para el modelo de SVM en el conjunto de TEST es: 0.7387 

La exactitud para el modelo de SVM en el conjunto de TRAIN es: 0.693 

Confusion Matrix
[[ 335 1844]
 [ 232 5535]] 

=============== Classification Report ================
              precision    recall  f1-score   support

         0.0       0.59      0.15      0.24      2179
         1.0       0.75      0.96      0.84      5767

    accuracy                           0.74      7946
   macro avg       0.67      0.56      0.54      7946
weighted avg       0.71      0.74      0.68      7946



In [12]:
matriz_confusion=pd.crosstab(y_test_bin,predict, rownames=['Clase predicha'], colnames=['Resultado del clasificador'])

In [13]:
matriz_confusion

Resultado del clasificador,0.0,1.0
Clase predicha,,
0.0,335,1844
1.0,232,5535


## 4. Clasficador SVM Multiclase
### 4.1 Creación del modelo y selección parámetros libres

In [14]:
clf = SVC()
modelo_base=clf.fit(X_train,y_train_multi)

C:\Users\rober\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


#### 3.2.1 Grid Search con validación cruzada

In [ ]:
#Cs = [1, 5]  #list(range(0,10)
#param_grid = {'kernel':('linear', 'rbf'),'C': Cs}
#svc_grid = GridSearchCV(clf, param_grid, scoring='accuracy',cv=3,n_jobs=-1)
#svc_grid.fit(X_train, y_train_multi)
#svc_grid.best_params_

svc_grid= SVC(C=5,kernel='linear',gamma='auto').fit(X_train, y_train_multi)  #quitar parentesis y eliminar este

In [ ]:
svc_clf=svc_grid

In [ ]:
print("Tasa de acierto del modelo base:",modelo_base.score(X_test,y_test_multi))
print("Tasa de acierto con la busqueda de parámetros:",svc_clf.score(X_test,y_test_multi))

#Mejora
print('Mejora  del {:0.2f}%.'.format( 100 * (svc_clf.score(X_test,y_test_multi) - modelo_base.score(X_test,y_test_multi)) / modelo_base.score(X_test,y_test_multi)))

### 4.3 Evaluación del modelo

In [ ]:
predict = svc_clf.predict(X_test)

#EVALUACION
acc_test= svc_clf.score(X_test, y_test_multi).round(4)
acc_train= svc_clf.score(X_train, y_train_multi).round(4)
print('La exactitud para el modelo de SVM en el conjunto de TEST es:',acc_test,'\n')
print('La exactitud para el modelo de SVM en el conjunto de TRAIN es:',acc_train,'\n')

print('Confusion Matrix')
print(confusion_matrix(y_test_multi, predict),'\n')

print("=============== Classification Report ================")
print(classification_report(y_test_multi, predict))

In [ ]:
matriz_confusion=pd.crosstab(y_test_multi,predict, rownames=['Clase predicha'], colnames=['Resultado del clasificador'])

In [ ]:
matriz_confusion

In [ ]:
pacientes = len(y_test_multi)
t = np.arange(0,pacientes)
plt.figure(figsize=(15,9))
plt.plot(t,y_test_multi,color='blue', marker='.',label='Real')
plt.plot(t,predict,color='green', marker='.',label='Predicha')

plt.xlabel('Pacientes')
plt.ylabel('Fase del sueño')
plt.title('Fase del sueño real vs. predicha en los primeros 30s')
plt.legend()

plt.title('Fase del sueño real vs. predicha en los primeros 30s')
plt.xlabel('Pacientes')
plt.ylabel('Fase del sueño')
plt.yticks([-1,0])
plt.xlim([0,10])
plt.legend()